# Data Exploration and Cleanup
----

### * Started with a 30 year suicide data file (kaggle via CDC(Centers for Disease Control))
### * Found and downloaded mental health facility locations from the samhsa.gov website
### * Found and dowloaded state population data from the Census Bureau
### * Found and dowloaded US mental health expenditures from KFF The Henry J. Kaiser Family Foundation website
### * Created an excel spreadsheet with suicide rates spanning years for many coutries from OEDC (Organization for economic cooperation and development)

#### The kaggle data required no specific cleanup and we immediately placed it in our cleandata folder.
#### The facility locations required some column manipulation, cleanup of one row with no latitude or longitude data, and removal of US territories.
#### The Census Bureau data required renaming of columns and creating a numeric column for populations.
#### The  KFF expenditure date required removing US territories, replacing "NR" (non reported) values with NaN and converting those affected columns to numeric values.
#### The OEDC data just required a conversion from excel to a .csv file.

####  All data was input from the rawdata folder, cleaned in this notebook and output to the cleandata folder.

---


In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_facility_data_file = "../data/cleandata/facilities.csv"
output_population_data_file = "../data/cleandata/population.csv"
output_expenditure_data_file = "../data/Expenditure.csv"

## Data Gathering
### * Behavioral Health Treatment Facilities (from Substance Abuse and Mental Health Service Administration)


In [2]:
#Mental Health Facilities in the US
facilities_file = "../data/rawdata/Behavioral_Health_Treatment_Facility_listing_2019_03_22_215606.csv"
facilities_df = pd.read_csv(facilities_file)

#Keep only necessary columns
facilities_df_sub = facilities_df[['    name1', 'name2', 'city', 'state', 'zip', 'latitude', 'longitude']]

#Suppress leading spaces on column header for name1
facilities_df_final = facilities_df_sub.rename(columns={"    name1":"name1"})

#Look at data for consistency
facilities_df_final.count()

name1        9686
name2        5430
city         9686
state        9686
zip          9686
latitude     9685
longitude    9685
dtype: int64

In [3]:
#drop one row with no latitude and longitude data
facilities_df_final.dropna(subset=['latitude', 'longitude'], inplace=True)
facilities_df_final.count()

name1        9685
name2        5429
city         9685
state        9685
zip          9685
latitude     9685
longitude    9685
dtype: int64

In [4]:
#remove us territories:  GU (Guam), AS (), PR (Puerto Rico), VI (Virgin Islands) 
#Note: keeping Washington DC in data, so total group of states will be 51
us_facilities_df = facilities_df_final[(facilities_df_final.state != 'GU') & (facilities_df_final.state != 'AS') & (facilities_df_final.state != 'PR') & (facilities_df_final.state != 'VI')]
print(us_facilities_df.count())
print(us_facilities_df.state.unique())

name1        9614
name2        5379
city         9614
state        9614
zip          9614
latitude     9614
longitude    9614
dtype: int64
['TX' 'LA' 'AR' 'MS' 'OK' 'AL' 'MO' 'TN' 'KS' 'NM' 'FL' 'IL' 'KY' 'GA'
 'CO' 'NE' 'IN' 'IA' 'NC' 'SC' 'AZ' 'VA' 'OH' 'SD' 'WI' 'WY' 'WV' 'MN'
 'UT' 'MI' 'PA' 'CA' 'MD' 'ND' 'NV' 'MT' 'DC' 'ID' 'NY' 'DE' 'NJ' 'CT'
 'MA' 'VT' 'OR' 'RI' 'NH' 'WA' 'ME' 'AK' 'HI']


In [5]:
# Export file as a CSV, without the Pandas index, but with the header
us_facilities_df.to_csv(output_facility_data_file, index=False, header=True)

## Data Gathering
### * US Population (from Census Bureau)

In [6]:
#US Census Bureau population in 50 US States
us_population_file = "../data/rawdata/PopulationEstimatesByState.csv"
pop_df = pd.read_csv(us_population_file)

#rename abbreviation column to state so as to be able to merge data with other files
pop_df = pop_df.rename(columns={"Abbv":"state",
                                "States":"State Name"})

#remove commans in population counts and make it a numeric field
pop_df['Population'] = pop_df['Population'].str.replace(',', '')
pop_df["Population"] = pd.to_numeric(pop_df["Population"])
print(pop_df.head())

## Export file as a CSV, without the Pandas index, but with the header
pop_df.to_csv(output_population_data_file, index=False, header=True)

  state  State Name  Population
0    AL     Alabama     4887871
1    AK      Alaska      737438
2    AZ     Arizona     7171646
3    AR    Arkansas     3013825
4    CA  California    39557045


## Data Gathering
### * US Expenditure

In [7]:
#Mental Health Expenditure 2004-2013 in the US

#----------------------------------------------------------------------------
expenditure_data_file = "../data/rawdata/StateMentalHealthServiceExpenditures2004_2013.csv"
expenditure_df = pd.read_csv(expenditure_data_file)

expenditure_df = expenditure_df[(expenditure_df.state != 'US') & (expenditure_df.state != 'PR')]

#replace NR values with np.nan
expenditure_df['FY2011__SMHA Expenditures Per Capita'] = expenditure_df['FY2011__SMHA Expenditures Per Capita'].replace('NR', np.NaN, regex=True)
expenditure_df['FY2013__SMHA Expenditures Per Capita'] = expenditure_df['FY2013__SMHA Expenditures Per Capita'].replace('NR', np.NaN, regex=True)

#change those columns to numeric values
expenditure_df["FY2011__SMHA Expenditures Per Capita"] = pd.to_numeric(expenditure_df["FY2011__SMHA Expenditures Per Capita"])
expenditure_df["FY2013__SMHA Expenditures Per Capita"] = pd.to_numeric(expenditure_df["FY2013__SMHA Expenditures Per Capita"])

expenditure_df


#-clean expenditure data-Complete. Dropped N/A, US &PR data,change NR data to NAN. 

,state,Location,FY2004__SMHA Expenditures Per Capita,FY2005__SMHA Expenditures Per Capita,FY2006__SMHA Expenditures Per Capita,FY2007__SMHA Expenditures Per Capita,FY2008__SMHA Expenditures Per Capita,FY2009__SMHA Expenditures Per Capita,FY2010__SMHA Expenditures Per Capita,FY2011__SMHA Expenditures Per Capita,FY2012__SMHA Expenditures Per Capita,FY2013__SMHA Expenditures Per Capita
1,AL,Alabama,58.78,60.31,64.43,74.03,79.39,77.89,78.19,78.33,76.27,72.64
2,AK,Alaska,287.56,269.64,278.63,293.16,277.03,289.71,310.01,380.33,335.36,341.08
3,AZ,Arizona,135.64,146.22,156.78,158.07,173.85,196.13,221.27,214.48,209.44,205.23
4,AR,Arkansas,32.95,35.60,38.58,39.72,40.53,42.77,42.02,NaN,45.05,45.56
5,CA,California,113.61,118.65,122.98,134.94,150.34,157.62,152.60,162.00,169.65,160.50
6,CO,Colorado,69.07,74.28,71.69,76.33,81.71,86.83,88.41,91.16,95.01,98.80
7,CT,Connecticut,152.09,157.21,169.68,174.94,188.74,197.62,189.34,202.44,213.43,216.76
8,DE,Delaware,84.92,89.19,103.60,105.75,111.07,109.13,106.04,101.94,100.02,96.53
9,DC,District of Columbia,409.92,404.40,394.10,392.27,381.90,388.83,NaN,307.45,305.37,306.87
10,FL,Florida,35.96,36.56,38.17,39.87,42.11,40.90,39.55,39.04,37.28,NaN


In [8]:
# Export file as a CSV, without the Pandas index, but with the header
expenditure_df.to_csv(output_expenditure_data_file, index=False, header=True)